In [2]:
import keras
keras.__version__

'2.2.0'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Using word embeddings

![word embeddings vs. one hot encoding](https://s3.amazonaws.com/book.keras.io/img/ch6/word_embeddings.png)

## 1 - Learning word embeddings with the `Embedding` layer

In [3]:
from keras.layers import Embedding

# The Embedding layer takes at least two arguments:
# the number of possible tokens, here 1000 (1 + maximum word index),
# and the dimensionality of the embeddings, here 64.
embedding_layer = Embedding(1000, 64)

### 1.1 - Prepare the data

Let's quickly prepare the data. We will restrict the movie reviews to the top 10,000 most common words, and cut the reviews after only 20 words. Our network will simply learn 8-dimensional embeddings for each of the 10,000 words, turn the input integer sequences (2D integer tensor) into embedded sequences (3D float tensor), flatten the tensor to 2D, and train a single `Dense` layer on top for classification.

In [1]:
from keras.datasets import imdb
from keras import preprocessing

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 20

# Load the data as lists of integers.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

Using TensorFlow backend.


In [4]:
assert(x_train.shape == (25000, 20))

In [5]:
x_train[0]

array([  65,   16,   38, 1334,   88,   12,   16,  283,    5,   16, 4472,
        113,  103,   32,   15,   16, 5345,   19,  178,   32], dtype=int32)

### 1.2 - Create the Embedding model

_QUESTION_ 

Create an architecture with the following elements : 
- Embedding layer : input_dim=10000, output_dim=8, input_length=maxlen
- Flatten
- Dense: units=1, activation="sigmoid"

Links to documentation

- Embedding : https://keras.io/layers/embeddings/
- Flatten : https://keras.io/layers/core/#flatten
- Dense : https://keras.io/layers/core/#dense

In [6]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

In [ ]:
model = Sequential()
# <YOUR CODE HERE>

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [12]:
assert(model.count_params() == 80161)

### 1.3 - Compile and fit the model

In [15]:
multi_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [17]:
history = multi_model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 46us/step - loss: 0.2771 - acc: 0.8903 - val_loss: 0.5231 - val_acc: 0.7516
Epoch 2/10
20000/20000 [==============================] - 1s 42us/step - loss: 0.2638 - acc: 0.8985 - val_loss: 0.5263 - val_acc: 0.7516
Epoch 3/10
20000/20000 [==============================] - 1s 45us/step - loss: 0.2508 - acc: 0.9050 - val_loss: 0.5315 - val_acc: 0.7502
Epoch 4/10
20000/20000 [==============================] - 1s 43us/step - loss: 0.2383 - acc: 0.9116 - val_loss: 0.5366 - val_acc: 0.7486
Epoch 5/10
20000/20000 [==============================] - 1s 44us/step - loss: 0.2261 - acc: 0.9182 - val_loss: 0.5428 - val_acc: 0.7466
Epoch 6/10
20000/20000 [==============================] - 1s 45us/step - loss: 0.2144 - acc: 0.9252 - val_loss: 0.5494 - val_acc: 0.7462
Epoch 7/10
20000/20000 [==============================] - 1s 45us/step - loss: 0.2032 - acc: 0.9296 - val_loss: 0.5570 - val_acc:

We get to a validation accuracy of ~76%, which is pretty good considering that we only look at the first 20 words in every review. 

## 2 - Using pre-trained word embeddings

Let's take a look at how you can get started using GloVe embeddings in a Keras model. The same method will of course be valid for Word2Vec embeddings or any other word embedding database that you can download.

### Putting it all together: from raw text to word embeddings


We will be using a model similar to the one we just went over -- embedding sentences in sequences of vectors, flattening them and training a 
`Dense` layer on top. But we will do it using pre-trained word embeddings, and instead of using the pre-tokenized IMDB data packaged in 
Keras, we will start from scratch, by downloading the original text data.

### 2.1 - Download the IMDB data as raw text


First, head to `http://ai.stanford.edu/~amaas/data/sentiment/` and download the raw IMDB dataset (if the URL isn't working anymore, just Google "IMDB dataset"). Uncompress it.

Now let's collect the individual training reviews into a list of strings, one string per review, and let's also collect the review labels 
(positive / negative) into a `labels` list:

In [18]:
import os

imdb_dir = '../../data/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [19]:
texts[0]

"This wasn't the major disaster that I was expecting, but that is about as positive as I can be in my description of the movie. I'm not sure what was meant to be funny about this movie, but I suppose it's all a matter of taste. Personally, I don't find it funny to watch morons living their idiotic lives or making fools of themselves on television, but then again, I'm not a fan of Jerry Springer's pathetic daytime talk show. I didn't get too bored watching this, but I was definitely never enjoying it, either. If you're in the mood to see a bad movie, but one that isn't too painful to sit through, this is a good choice."

### 2.2 - Tokenize the data


Let's vectorize the texts we collected, and prepare a training and validation split.
We will merely be using the concepts we introduced earlier in this section.

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words
training_samples = 200  # We will be training on 200 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

Documentation : https://keras.io/preprocessing/text/#tokenizer

Write the following steps : 
- Instanciate a Tokenizer with *num_words=max_words*
- Use the method *fit_on_texts*
- Use the method *texts_to_sequences* to transform the sequences

In [ ]:
# Tokenizer instanciation
tokenizer = # <YOUR CODE HERE>

In [ ]:
# Fit Tokenizer
# <YOUR CODE HERE>

In [ ]:
# Transform texts to sequences
sequences = # <YOUR CODE HERE>

In [24]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 88582 unique tokens.


In [25]:
# Pad sequences
data = pad_sequences(sequences, maxlen=maxlen)

In [28]:
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [29]:
# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

### 2.3 - Download the GloVe word embeddings


Head to `https://nlp.stanford.edu/projects/glove/` (where you can learn more about the GloVe algorithm), and download the pre-computed embeddings from 2014 English Wikipedia. It's a 822MB zip file named `glove.6B.zip`, containing 100-dimensional embedding vectors for 400,000 words (or non-word tokens). Un-zip it.

### 2.4 - Pre-process the embeddings


Let's parse the un-zipped file (it's a `txt` file) to build an index mapping words (as strings) to their vector representation (as number 
vectors).

In [30]:
glove_dir = '../../data/glove.6B/'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


Now let's build an embedding matrix that we will be able to load into an `Embedding` layer. It must be a matrix of shape `(max_words, embedding_dim)`, where each entry `i` contains the `embedding_dim`-dimensional vector for the word of index `i` in our reference word index (built during tokenization). Note that the index `0` is not supposed to stand for any word or token -- it's a placeholder.

In [31]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [33]:
embedding_matrix[1]

array([-0.038194  , -0.24487001,  0.72812003, -0.39961001,  0.083172  ,
        0.043953  , -0.39140999,  0.3344    , -0.57545   ,  0.087459  ,
        0.28786999, -0.06731   ,  0.30906001, -0.26383999, -0.13231   ,
       -0.20757   ,  0.33395001, -0.33848   , -0.31742999, -0.48335999,
        0.1464    , -0.37303999,  0.34577   ,  0.052041  ,  0.44946   ,
       -0.46970999,  0.02628   , -0.54154998, -0.15518001, -0.14106999,
       -0.039722  ,  0.28277001,  0.14393   ,  0.23464   , -0.31020999,
        0.086173  ,  0.20397   ,  0.52623999,  0.17163999, -0.082378  ,
       -0.71787   , -0.41531   ,  0.20334999, -0.12763   ,  0.41367   ,
        0.55186999,  0.57907999, -0.33476999, -0.36559001, -0.54856998,
       -0.062892  ,  0.26583999,  0.30204999,  0.99774998, -0.80480999,
       -3.0243001 ,  0.01254   , -0.36941999,  2.21670008,  0.72201002,
       -0.24978   ,  0.92136002,  0.034514  ,  0.46744999,  1.10790002,
       -0.19358   , -0.074575  ,  0.23353   , -0.052062  , -0.22

### 2.5 - Define a model

_QUESTION_ 

Create an architecture with the following elements : 
- Embedding layer : input_dim=max_words, output_dim=embedding_dim, input_length=maxlen
- Flatten
- Dense: units=32, activation="relu"
- Dense: units=1, activation="sigmoid"

Links to documentation

- Embedding : https://keras.io/layers/embeddings/
- Flatten : https://keras.io/layers/core/#flatten
- Dense : https://keras.io/layers/core/#dense

In [34]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
# <YOUR CODE HERE>

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          1000000   
Total params: 1,000,000
Trainable params: 1,000,000
Non-trainable params: 0
_________________________________________________________________


In [ ]:
assert(model.count_params() == 1320065)

### 2.6 - Load the GloVe embeddings in the model


The `Embedding` layer has a single weight matrix: a 2D float matrix where each entry `i` is the word vector meant to be associated with index `i`. Simple enough. Let's just load the GloVe matrix we prepared into our `Embedding` layer, the first layer in our model:

_QUESTION_

- Use the `set_weights` method of the first layer to load the prepared weights embedding_matrix (the argument must be a list of matrices - one in our case)
- Set the `trainable` parameter of this first layer to False

In [ ]:
# Set weights of the first layer
# <YOUR CODE HERE>
# Set first layer parameters to not trainable
# <YOUR CODE HERE>

In [ ]:
model.summary()

> Your model must now have 1 000 000 non trainable parameters

### 2.7 - Compile and Train the model

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))

In [ ]:
model.save_weights('../../models/pre_trained_glove_model.h5')

Let's plot its performance over time:

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

The model quickly starts overfitting, unsurprisingly given the small number of training samples. Validation accuracy has high variance for the same reason, but seems to reach high 50s.

Note that your mileage may vary: since we have so few training samples, performance is heavily dependent on which exact 200 samples we picked, and we picked them at random. If it worked really poorly for you, try picking a different random set of 200 samples, just for the sake of the exercise (in real life you don't get to pick your training data).